In [3]:
import geopandas as gpd
from pathlib import Path
import hvplot.pandas
import pandas as pd
import holoviews as hv
import geoviews as gv
import numpy as np
import os

hv.extension('bokeh')

## Select the reservoir

In [4]:
RESERVOIR = '0810'

In [5]:
# read the bounding box of the study area
val_pts = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-pts.geojson'))
val_polys = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-poly.geojson'))

selected_reservoirs = val_pts['tmsos_id'].tolist()  # select all 100 reservoirs
res_names = val_pts[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name'] # dictionary that can be queried to get reservoir name

RESERVOIR_NAME = res_names[RESERVOIR]

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

# get reservoir properties from GRanD
reservoir = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]
nominal_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_SKM'].values[0]
nominal_area_poly = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_POLY'].values[0]
max_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MAX'].values[0]
max_area = np.nan if max_area == -99 else max_area

min_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MIN'].values[0]
min_area = 0 if min_area == -99 else min_area

area_rep = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_REP'].values[0]
dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DAM_HGT_M'].values[0])
elev_msl = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['ELEV_MASL'].values[0])
depth = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DEPTH_M'].values[0])
capacity = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['CAP_MCM'].values[0])


## Plot a map of the selected reservoirs
global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM',
        hover_cols=['tmsos_id', 'name'],
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
        hover_cols=['tmsos_id', 'name'],
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red",
    hover_tooltips=[('tmsos_id', '@tmsos_id'), ('name', '@name')],
)

print(
    f"Selected reservoir: {RESERVOIR}: {RESERVOIR_NAME}\n",
    f"{nominal_area = }\n",
    f"{nominal_area_poly = }\n",
    f"{max_area = }\n",
    f"{min_area = }\n",
    f"{area_rep = }\n",
    f"{dam_height = }\n",
    f"{elev_msl = }\n",
    f"{depth = }\n",
    f"{capacity = }\n",
)

global_map

Selected reservoir: 0810: Noi, Th
 nominal_area = 235.58
 nominal_area_poly = 235.58
 max_area = nan
 min_area = 230.0
 area_rep = 288.3
 dam_height = 42.0
 elev_msl = 149.0
 depth = 8.3
 capacity = 1966.0



:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (tmsos_id,name)
   .Points.II :Points   [Longitude,Latitude]   (tmsos_id,name)

In [6]:
# what is the reported capacity?
capacity_hv = hv.HLine(capacity).opts(color='red', ylim=(0, capacity + capacity*0.1), ylabel='capacity (Mil. m3)')
capacity_hv

:HLine   [x,y]

In [7]:
def get_insitu_observed_data(tmsos_id, val_polys, deltares_insitu_dir=Path('/tiger1/pdas47/tmsosPP/data/insitu/deltares'), rid_insitu_dir=Path('/tiger1/pdas47/tmsosPP/data/insitu/rid'), resops_insitu_dir=Path('/tiger1/pdas47/tmsosPP/data/insitu/resopsus'), rise_insitu_dir=Path('/tiger1/pdas47/tmsosPP/data/insitu/rise')):
    row = val_polys[val_polys['tmsos_id'] == tmsos_id]
    db = row['db'].values[0]

    insitu_df = None
    if db == 'rise':
        rise_id = row['rise_id'].item()
        storage_fn = rise_insitu_dir / rise_id / 'Lake_Reservoir_Storage.csv'
        insitu_df = pd.read_csv(storage_fn, parse_dates=['Datetime (UTC)'])
        assert insitu_df['Units'].iloc[0] == 'af'
        insitu_df['Result'] = insitu_df['Result'] * 1233.48183
        insitu_df['date'] = pd.to_datetime(insitu_df['Datetime (UTC)'].dt.date)
        insitu_df = insitu_df.rename({'Result': 'observed storage [m3]'}, axis=1)
        columns_to_drop = ['Unnamed: 0', 'Location', 'Parameter', 'Units', 'Timestep', 'Aggregation', 'timeStep', 'resultType']
        insitu_df = insitu_df.drop(columns=[col for col in columns_to_drop if col in insitu_df.columns])
        insitu_df['observed storage [Mm3]'] = insitu_df['observed storage [m3]'] * 1e-6
        insitu_df['db'] = 'rise'

        area_fn = rise_insitu_dir / rise_id / 'Lake_Reservoir_Area.csv'
        if area_fn.exists():
            insitu_area_df = pd.read_csv(area_fn, parse_dates=['Datetime (UTC)'])
            assert insitu_area_df['Units'].iloc[0] == 'acres'
            insitu_area_df['Result'] = insitu_area_df['Result'] * 0.0040468564
            insitu_area_df['date'] = pd.to_datetime(insitu_area_df['Datetime (UTC)'].dt.date)
            insitu_area_df = insitu_area_df.rename({'Result': 'observed area [km2]'}, axis=1)
            columns_to_drop = ['Unnamed: 0', 'Location', 'Parameter', 'Units', 'Timestep', 'Aggregation', 'timeStep', 'resultType']
            insitu_area_df = insitu_area_df.drop(columns=[col for col in columns_to_drop if col in insitu_area_df.columns])
            insitu_df = insitu_df.merge(insitu_area_df, on='date', how='left')

        elevation_fn = rise_insitu_dir / rise_id / 'Lake_Reservoir_Elevation.csv'
        if elevation_fn.exists():
            insitu_elevation_df = pd.read_csv(elevation_fn, parse_dates=['Datetime (UTC)'])
            assert insitu_elevation_df['Units'].iloc[0] == 'ft'
            insitu_elevation_df['Result'] = insitu_elevation_df['Result'] * 0.3048
            insitu_elevation_df['date'] = pd.to_datetime(insitu_elevation_df['Datetime (UTC)'].dt.date)
            insitu_elevation_df = insitu_elevation_df.rename({'Result': 'observed wse [m]'}, axis=1)
            columns_to_drop = ['Unnamed: 0', 'Location', 'Parameter', 'Units', 'Timestep', 'Aggregation', 'timeStep', 'resultType']
            insitu_elevation_df = insitu_elevation_df.drop(columns=[col for col in columns_to_drop if col in insitu_elevation_df.columns])
            insitu_df = insitu_df.merge(insitu_elevation_df, on='date', how='left')

    elif db == 'deltares':
        deltares_id = row['deltares_id'].values[0]
        fn = deltares_insitu_dir / f'{int(deltares_id):07}.csv'
        insitu_df = pd.read_csv(fn, parse_dates=['time'])
        insitu_df.sort_values('time', inplace=True)
        insitu_df['date'] = pd.to_datetime(insitu_df['time'].dt.date)
        insitu_df['observed area [km2]'] = insitu_df['area'] * 1e-6
        insitu_df['db'] = 'deltares'
        insitu_df = insitu_df[['date', 'observed area [km2]', 'db']]

    elif db == 'rid':
        rid_filename = row['rid_filename'].values[0]
        fn = rid_insitu_dir / rid_filename
        insitu_df = pd.read_csv(fn, parse_dates=['date'])
        insitu_df['observed wse [m]'] = insitu_df['water_level (m)']
        insitu_df['observed storage [Mm3]'] = insitu_df['storage (mil. m3)']
        insitu_df['db'] = 'rid'
        insitu_df = insitu_df[['date', 'observed wse [m]', 'observed storage [Mm3]', 'db']]

    elif db == 'resops':
        resops_id = int(row['resops_id'].values[0])
        fn = resops_insitu_dir / f'ResOpsUS_{resops_id}.csv'
        insitu_df = pd.read_csv(fn, parse_dates=['date'])
        insitu_df['observed wse [m]'] = insitu_df['elevation']
        insitu_df['observed storage [Mm3]'] = insitu_df['storage']
        insitu_df['db'] = 'resops'
        insitu_df = insitu_df[['date', 'observed wse [m]', 'observed storage [Mm3]', 'db']]

    return insitu_df

observed_df = get_insitu_observed_data(RESERVOIR, val_res_poly)
observed_df

/tmp/ipykernel_2196354/538103603.py:54: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  insitu_df = pd.read_csv(fn, parse_dates=['date'])


,date,observed wse [m],observed storage [Mm3],db
0,1980-01-01,139.960458,1429.12,rid
1,1980-01-02,139.944712,1425.20,rid
2,1980-01-03,139.928934,1421.29,rid
3,1980-01-04,139.913125,1417.38,rid
4,1980-01-05,139.898076,1413.47,rid
...,...,...,...,...
16043,2024-03-24,139.310000,1236.71,rid
16044,2024-03-25,139.290000,1232.36,rid
16045,2024-03-26,139.280000,1230.19,rid
16046,2024-03-27,139.270000,1228.02,rid


In [8]:
observed_df = observed_df[observed_df['observed storage [Mm3]'] > 0]

In [9]:
observed_df.hvplot(x='observed wse [m]', y='observed storage [Mm3]', kind='scatter', color='blue', title=f'Observed storage and water surface elevation for {RESERVOIR_NAME}')

:Scatter   [observed wse [m]]   (observed storage [Mm3])

In [10]:
# Fit a 2-degree polynomial
coefficients = np.polyfit(observed_df['observed wse [m]'], observed_df['observed storage [Mm3]'], 2)
polynomial = np.poly1d(coefficients)

# Generate x values for plotting the polynomial
x_values = np.linspace(observed_df['observed wse [m]'].min(), observed_df['observed wse [m]'].max(), 100)
y_values = polynomial(x_values)

# Plot the observed data and the polynomial fit
plot = observed_df.hvplot.scatter(
    x='observed wse [m]', y='observed storage [Mm3]', color='blue', title=f'Observed storage and water surface elevation for {RESERVOIR_NAME}'
).opts(width=700, height=500) * hv.Curve((x_values, y_values), label='2-degree polynomial fit').opts(color='red')
plot

:Overlay
   .Scatter.I                                    :Scatter   [observed wse [m]]   (observed storage [Mm3])
   .Curve.A_2_hyphen_minus_degree_polynomial_fit :Curve   [x]   (y)

In [11]:
# Fit polynomial curves to predict storage from elevation and area
coefficients_elevation = np.polyfit(observed_df['observed wse [m]'], observed_df['observed storage [Mm3]'], 2)
polynomial_elevation = np.poly1d(coefficients_elevation)

# Generate equally spaced elevation values
min_elevation = np.floor(observed_df['observed wse [m]'].min() / 0.25) * 0.25
max_elevation = np.ceil(observed_df['observed wse [m]'].max() / 0.25) * 0.25
elevation_values = np.arange(min_elevation, max_elevation + 0.25, 0.25)

# Predict storage values using the polynomial functions
predicted_storage_elevation = polynomial_elevation(elevation_values)

# Interpolate the observed storage using linear interpolation for the elevation_values
observed_storage_interpolated = np.interp(elevation_values, observed_df['observed wse [m]'], observed_df['observed storage [Mm3]'])

# Create a dataframe with the predicted storage, interpolated observed storage, and elevation values
predicted_df = pd.DataFrame({
    'Elevation': elevation_values,
    'Storage (mil. m3)': predicted_storage_elevation,
})

# Save the polynomial functions as comments
polynomial_elevation_str = f"Polynomial function (Elevation): {polynomial_elevation}"

# Display the dataframe and polynomial functions
print(polynomial_elevation_str)
predicted_df

Polynomial function (Elevation):        2
11.12 x - 2868 x + 1.849e+05


,Elevation,Storage (mil. m3)
0,137.00,794.222391
1,137.25,839.823400
2,137.50,886.814552
3,137.75,935.195847
4,138.00,984.967286
5,138.25,1036.128868
6,138.50,1088.680593
7,138.75,1142.622461
8,139.00,1197.954472
9,139.25,1254.676627


In [12]:
output_dir = Path('../data/aec/aev_insitu')
output_dir.mkdir(parents=True, exist_ok=True)  # Create the directory if it doesn't exist

output_file = output_dir / f'{RESERVOIR}.csv'
predicted_df.to_csv(output_file, index=False)